In [35]:
import os
import json
import pandas as pd

# User
file_path = os.path.join('accounts/fixtures/accounts', 'fake_user.json')
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

df_data = {
    'birth': [],
    'gender': [],
    'crdt_grad': [],
    'salary': [],
    'm_consumption':[],
    'asset':[],
    'real_estate':[],
    'invest_tendency':[],
    'pk':[],
}

for i in data:
    for pro in i['fields']['product']:
        df_data['birth'].append(i['fields']['birth'])
        df_data['gender'].append(i['fields']['gender'])
        df_data['crdt_grad'].append(i['fields']['crdt_grad'])
        df_data['salary'].append(i['fields']['salary'])
        df_data['m_consumption'].append(i['fields']['m_consumption'])
        df_data['asset'].append(i['fields']['asset'])
        df_data['real_estate'].append(i['fields']['real_estate'])
        df_data['invest_tendency'].append(i['fields']['invest_tendency'])
        df_data['pk'].append(pro)
user = pd.DataFrame(df_data)

# Deposit products
file_path = os.path.join('savedata/fixtures/savedata', 'deposit_products.json')
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
df_data = {
    'pk': [],               # pk
    'company': [],        # 금융회사코드
    'product_number': [],      # 금융상품코드
    'join_deny': [],        # 가입 제한
    'join_member': [],      # 가입 대상
    'join_way': [],         # 가입 방법
    'type':[],      # 예금/적금
}

for i in data:
    df_data['pk'].append(i['pk'])
    df_data['company'].append(i['fields']['fin_co_no'])
    df_data['product_number'].append(i['fields']['fin_prdt_cd'])
    df_data['join_deny'].append(i['fields']['join_deny'])
    df_data['join_member'].append(i['fields']['join_member'])
    df_data['join_way'].append(i['fields']['join_way'])
    df_data['type'].append(i['fields']['product_type'])
products = pd.DataFrame(df_data)

# Deposit options
file_path = os.path.join('savedata/fixtures/savedata', 'deposit_options.json')
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
df_data = {
    'company': [],
    'product_number': [],
    'intr_rate_type_nm': [],
    'intr_rate': [],
    'intr_rate2': [],
    'save_trm':[],
    'rsrv_type_nm':[],
    'rsrv_type':[],
}

for i in data:
    df_data['company'].append(i['fields']['fin_co_no'])
    df_data['product_number'].append(i['fields']['fin_prdt_cd'])
    df_data['intr_rate_type_nm'].append(i['fields']['intr_rate_type_nm'])
    df_data['intr_rate'].append(i['fields']['intr_rate'])
    df_data['intr_rate2'].append(i['fields']['intr_rate2'])
    df_data['save_trm'].append(i['fields']['save_trm'])
    df_data['rsrv_type_nm'].append(i['fields']['rsrv_type_nm'])
    df_data['rsrv_type'].append(i['fields']['rsrv_type'])
options = pd.DataFrame(df_data)

In [30]:
display(user.head())
display(user.isnull().sum())
print()
display(products.head())
display(products.isnull().sum())
print()
display(options.head())
display(options.isnull().sum())

,birth,gender,crdt_grad,salary,m_consumption,asset,real_estate,invest_tendency,product
0,1955-11-19,1,874,4412,851,19667,1,2,184
1,1955-11-19,1,874,4412,851,19667,1,2,379
2,1955-11-19,1,874,4412,851,19667,1,2,400
3,1992-12-12,1,361,20297,266,29320,0,3,325
4,1992-12-12,1,361,20297,266,29320,0,3,429


birth              0
gender             0
crdt_grad          0
salary             0
m_consumption      0
asset              0
real_estate        0
invest_tendency    0
product            0
dtype: int64

,pk,company,product_number,join_deny,join_member,join_way,type
0,1,0010001,WR0001B,1,실명의 개인,"인터넷,스마트폰,전화(텔레뱅킹)",예금
1,2,0010002,00320342,1,개인(개인사업자 포함),"인터넷,스마트폰",예금
2,3,0010016,10511008000996000,1,실명의 개인,"영업점,인터넷,스마트폰",예금
3,4,0010016,10511008001004000,3,만50세 이상 개인,"영업점,인터넷,스마트폰",예금
4,5,0010016,10511008001166004,1,실명의 개인 및 개인사업자,"영업점,인터넷,스마트폰",예금


pk                0
company           0
product_number    0
join_deny         0
join_member       0
join_way          0
type              0
dtype: int64

,company,product_number,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type_nm,rsrv_type
0,0010001,WR0001B,단리,3.00,3.00,1,None,None
1,0010001,WR0001B,단리,3.50,3.50,3,None,None
2,0010001,WR0001B,단리,3.50,3.50,6,None,None
3,0010001,WR0001B,단리,3.55,3.55,12,None,None
4,0010001,WR0001B,단리,3.00,3.00,24,None,None


company                 0
product_number          0
intr_rate_type_nm       0
intr_rate               0
intr_rate2              0
save_trm                0
rsrv_type_nm         2782
rsrv_type            2782
dtype: int64

In [36]:
pro = products.merge(options)
pro.head(5)

,pk,company,product_number,join_deny,join_member,join_way,type,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type_nm,rsrv_type
0,1,0010001,WR0001B,1,실명의 개인,"인터넷,스마트폰,전화(텔레뱅킹)",예금,단리,3.00,3.00,1,None,None
1,1,0010001,WR0001B,1,실명의 개인,"인터넷,스마트폰,전화(텔레뱅킹)",예금,단리,3.50,3.50,3,None,None
2,1,0010001,WR0001B,1,실명의 개인,"인터넷,스마트폰,전화(텔레뱅킹)",예금,단리,3.50,3.50,6,None,None
3,1,0010001,WR0001B,1,실명의 개인,"인터넷,스마트폰,전화(텔레뱅킹)",예금,단리,3.55,3.55,12,None,None
4,1,0010001,WR0001B,1,실명의 개인,"인터넷,스마트폰,전화(텔레뱅킹)",예금,단리,3.00,3.00,24,None,None


In [37]:
user.merge(pro, on="pk")

,birth,gender,crdt_grad,salary,m_consumption,asset,real_estate,invest_tendency,pk,company,...,join_deny,join_member,join_way,type,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type_nm,rsrv_type
0,1955-11-19,1,874,4412,851,19667,1,2,184,0010464,...,1,내·외국인 대상,인터넷,예금,단리,1.2,1.2,1,None,None
1,1955-11-19,1,874,4412,851,19667,1,2,184,0010464,...,1,내·외국인 대상,인터넷,예금,단리,3.0,3.0,3,None,None
2,1955-11-19,1,874,4412,851,19667,1,2,184,0010464,...,1,내·외국인 대상,인터넷,예금,단리,3.6,3.6,6,None,None
3,1955-11-19,1,874,4412,851,19667,1,2,184,0010464,...,1,내·외국인 대상,인터넷,예금,단리,3.9,3.9,12,None,None
4,1955-11-19,1,874,4412,851,19667,1,2,184,0010464,...,1,내·외국인 대상,인터넷,예금,단리,3.0,3.0,24,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16926,1966-08-24,1,926,18423,842,48219,1,0,322,0010556,...,1,제한없음,영업점,예금,단리,1.6,1.6,36,None,None
16927,1987-10-15,0,820,7086,678,67593,0,1,566,0010453,...,1,개인,스마트폰,적금,단리,2.5,2.5,6,정액적립식,S
16928,1987-10-15,0,820,7086,678,67593,0,1,566,0010453,...,1,개인,스마트폰,적금,단리,4.0,4.0,12,정액적립식,S
16929,1987-10-15,0,820,7086,678,67593,0,1,566,0010453,...,1,개인,스마트폰,적금,단리,3.4,3.4,24,정액적립식,S
